```{eval-rst}
.. include:: sinebow.rst

```
{sinebow23}`Logo`
==================

This is a little cute: Omnipose can "segment" text using the `bact_phase_omni` model. Semantic segmentation of uniform, 
disjoint shapes on a uniform background is absolutely no feat, but it is amusing that a neural network trained purely on 
phase contrast images of bacteria gives such reasonable output on something so different from the training set. 
Also, the over-segmentation at cusps hints that the network has learned to pick up on local morphology. 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from omnipose.utils import bbox_to_slice

tsizes = [60]
texts = ["Omnipose","O"]
imgs = []
for textsize in tsizes:
    fonts = [ImageFont.truetype(f, textsize) for f in ["SFNSRounded.ttf"]]
    for text in texts:
        for font in fonts:
            size = np.array([textsize*len(text)*2, textsize*2])
            im = Image.new("RGB", tuple(size), "white")
            d = ImageDraw.Draw(im)
            center = size/2
            anchor = "mm"
            d.text(center, text, fill="black", anchor=anchor, font=font)

            bbox = d.textbbox(center, text, anchor=anchor, font=font)
            bbox
            bbox = [bbox[1],bbox[0],bbox[3],bbox[2]] # reverse x, y
            im = np.array(im)
            slc = bbox_to_slice(bbox,im.shape,pad = 3)
            im = im[slc]
            imgs.append(im)
            
            fig = plt.figure(figsize=(1,)*2)
            plt.imshow(im)
            plt.axis('off')
            plt.show()

In [ ]:
from cellpose import models, core
model_name = 'bact_phase_omni'
use_GPU = core.use_gpu()
model = models.CellposeModel(gpu=use_GPU, model_type=model_name)


chans = [0,0] #this means segment based on first channel, no second channel 
nimg = len(imgs)
n = range(nimg) 

# define parameters
mask_threshold = 1
verbose = 0 
use_gpu = use_GPU 
transparency = True 
rescale=None
omni = True 
flow_threshold = 0 
resample = True 
cluster = False

masks, flows, styles = model.eval([imgs[i] for i in n],
                                  channels=chans,
                                  rescale=rescale,
                                  mask_threshold=mask_threshold,
                                  transparency=transparency,
                                  flow_threshold=flow_threshold,
                                  omni=omni,resample=resample,
                                  verbose=verbose, 
                                  cluster=cluster)

In [ ]:
from cellpose import plot
import omnipose

for idx,i in enumerate(n):

    maski = masks[idx]
    flowi = flows[idx][0]

    fig = plt.figure(figsize=(12,5))
    plot.show_segmentation(fig, omnipose.utils.normalize99(imgs[i]), maski, flowi, channels=chans, omni=True, bg_color=0)
    
    plt.tight_layout()
    plt.show()

I landed on this font because it is one of Apple's sytem defaults (and thefore works well with the system fonts used on our website when viewed on Apple devices), and I chose this scale because it is very close to bacteria and showed a good amount of 'segmentation' in the M, N, P, and E from purely local morphology (cusps). This gives reasonable output at higher-resulution text (wider 'cells'), but it starts to hallucinate output between objects if the size gets too large. 

The transparency (alpha channel) is set by the flow magnitude, and the color (RGB channels) is set by the flow angle according to a shifted sinebow relation:
```python
angles = np.arctan2(dP[1], dP[0])+np.pi
r = ((np.cos(angles)+1)/2)
g = ((np.cos(angles+2*np.pi/3)+1)/2)
b =((np.cos(angles+4*np.pi/3)+1)/2)
```
(a is just a constant, not alpha). The slight tinge of green comes from the fact that `np.arctan2(0,0)=0` and `((np.cos(0+2*np.pi/3)+1)/2) = 1/4`. I'll try two ways to remove it: 
first by removing any average background bias, second by adjusting the alpha channel so that the background alpha is 0 on average. 

In [ ]:
import skimage.io
import os
from omnipose.utils import normalize99, rescale
from scipy.ndimage import zoom

basedir = '_static/'
names = ['logo.png','icon.ico']
ext = '.png'
for idx,i in enumerate(n):

    maski = masks[idx]
    flowi = flows[idx][0]
    dPi = flows[idx][1]
    bias = [np.mean(d[maski==0]) for d in dPi]
    angle = np.arctan2(bias[1], bias[0]) / np.pi
    print('avarage bias is {}, average angle is {} pi rad.'.format(bias,angle))
    dPi_new = np.stack([np.clip(d - b,-np.inf,np.inf) for d,b in zip(dPi,bias)])
    flowi_new = plot.dx_to_circ(dPi_new,transparency=True)
    
    flowi_3 = flowi.copy()
    alpha = flowi_3[...,-1]
    flowi_3[...,-1] = rescale(np.clip(alpha-np.mean(alpha[maski==0]),0,np.inf))*255
    
    
    plt.imshow(np.hstack([flowi,flowi_new,flowi_3]))
    plt.axis('off')
    plt.show()
    # aplha channel correction is the winner
    # also rescale the image without interpolation so that, when displayed as favicon etc., it is not as smoothed out - we want to show real output 
    skimage.io.imsave(os.path.join(basedir,names[idx]),zoom(flowi_3,(3,)*(flowi.ndim-1)+(1,),order=0)) 
    

Turns out that subtracting off the flow component bias introduces some overcorrection in places, leading to some discoloration. So, alpha adjustment it is. It might be hard for you to see it, but I can. This level of pixel-peeping is how I made my ground-truth data ;-) 